In [1]:
# import useful libraries

import numpy as np
import itertools
from collections import Counter
from utils import load_puzzle_input, test_code

In [2]:
# load puzzle input
    
puzzle_input = load_puzzle_input('input.txt')

print(puzzle_input[:3])

['D 1', 'L 2', 'D 2']


### Part 1

In [3]:
# code for preparing and testing examples for part 1

example_dict_part_1 = {
    tuple(load_puzzle_input('example_1.txt')): 13
}

In [4]:
def calulate_distance(x_h, y_h, x_t, y_t):
    return max(
        abs(
            x_h - x_t
            ), 
        abs(
            y_h - y_t
            )
    )

In [30]:
def determine_T_movement(x_h, y_h, x_t, y_t):

    # vertical movement only
    if x_h == x_t:
        if y_h > y_t:
            y_t += 1
        else:
            y_t -= 1
            
    # horizontal movement only
    elif y_h == y_t:
        if x_h > x_t:
            x_t += 1
        else:
            x_t -= 1
    
    # diagonal movement
    else:
        if y_h > y_t:
            y_t += 1
        elif y_h < y_t:
            y_t -= 1
            
        if x_h > x_t:
            x_t += 1
        elif x_h < x_t:
            x_t -= 1

    return x_t, y_t

In [14]:
def visualise_grid(x_h, y_h, x_t, y_t, shape = (6, 5)):
    """For debugging only"""
    
    grid = np.array(
        [['.' for _ in range(shape[0])] for _ in range(shape[1])]
    )

    grid[shape[1] - 0 - 1][0] = 's'
    grid[shape[1] - y_t - 1][x_t] = 'T'
    grid[shape[1] - y_h - 1][x_h] = 'H'
    for row in grid:
        print(' '.join(row))

In [35]:
def resolve_instruction(direction_str, n_steps, x_h, y_h, x_t, y_t, visited_set, verbose = False, shape = (6, 5)):
    
    if verbose:
        print(f"== {direction_str} {n_steps} ==")
        print()

    for _ in range(n_steps): 

        if direction_str == 'U':
            y_h += 1
        elif direction_str == 'R':
            x_h += 1
        elif direction_str == 'D':
            y_h -= 1
        elif direction_str == 'L':
            x_h -= 1
            
        # print(x_h, y_h, x_t, y_t)

        if calulate_distance(x_h, y_h, x_t, y_t) > 2:
            print('ERROR')
            print(x_h, y_h, x_t, y_t)
            raise ValueError

        if calulate_distance(x_h, y_h, x_t, y_t) > 1:
            x_t, y_t = determine_T_movement(x_h, y_h, x_t, y_t)
            
            visited_set.add((x_t, y_t))

        if verbose:
            visualise_grid(x_h, y_h, x_t, y_t)
            print()

    return x_h, y_h, x_t, y_t, visited_set

In [36]:
def part_1_solution(puzzle_input, verbose = False):

    visited_set = {(0, 0)}
    x_h, y_h, x_t, y_t = 0, 0, 0, 0

    for instruction_str in puzzle_input:
        
        # print(instruction_str)

        direction_str, n_steps = instruction_str.split()
        n_steps = int(n_steps)
        
        # print(f"{direction_str} : {n_steps}")
        # print()

        x_h, y_h, x_t, y_t, visited_set = resolve_instruction(direction_str, n_steps, x_h, y_h, x_t, y_t, visited_set, verbose = verbose)

    return len(visited_set)

In [37]:
# test part 1 solution

test_code(part_1_solution, example_dict_part_1)

Test 0 passed: Input <('R 4', 'U 4', 'L 3', 'D 1', 'R 4', 'D 1', 'L 5', 'R 2')> gives output <13>.

Congratulations! Looks like you cracked it! Good job!


In [39]:
part_1_solution(puzzle_input)

6745

### Part 2

In [45]:
# code for preparing and testing examples for part 2

example_dict_part_2 = {
    tuple(load_puzzle_input('example_1.txt')): 1,
    tuple(load_puzzle_input('example_2.txt')): 36,
}

In [64]:
def resolve_instruction_2(direction_str, n_steps, x_h, y_h, mid_knots, x_t, y_t, visited_set, verbose = False, shape = (6, 5)):
    
    if verbose:
        print(f"== {direction_str} {n_steps} ==")
        print()

    for _ in range(n_steps): 

        if direction_str == 'U':
            y_h += 1
        elif direction_str == 'R':
            x_h += 1
        elif direction_str == 'D':
            y_h -= 1
        elif direction_str == 'L':
            x_h -= 1
            
        # print(x_h, y_h, x_t, y_t)
        
        x_prev, y_prev = x_h, y_h

        for idx, (x_n, y_n) in enumerate(mid_knots):
            
            if calulate_distance(x_prev, y_prev, x_n, y_n) > 1:
                mid_knots[idx] = determine_T_movement(x_prev, y_prev, x_n, y_n)

            x_prev, y_prev = mid_knots[idx]
            
        # print(mid_knots)
            
        if calulate_distance(x_prev, y_prev, x_t, y_t) > 1:
            x_t, y_t = determine_T_movement(x_prev, y_prev, x_t, y_t)
                
        visited_set.add((x_t, y_t))

        # if verbose:
        #     visualise_grid(x_h, y_h, x_t, y_t)
        #     print()

    return x_h, y_h, mid_knots, x_t, y_t, visited_set

In [68]:
def part_2_solution(puzzle_input):
    
    visited_set = {(0, 0)}
    x_h, y_h, x_t, y_t = 0, 0, 0, 0
    mid_knots = [(0,0) for _ in range(8)]

    for instruction_str in puzzle_input:
        
        # print(instruction_str)

        direction_str, n_steps = instruction_str.split()
        n_steps = int(n_steps)
        
        # print(f"{direction_str} : {n_steps}")
        # print()

        x_h, y_h, mid_knots, x_t, y_t, visited_set = resolve_instruction_2(direction_str, n_steps, x_h, y_h, mid_knots, x_t, y_t, visited_set)

    return len(visited_set)

In [69]:
# test part 2 solution

test_code(part_2_solution, example_dict_part_2)

Test 0 passed: Input <('R 4', 'U 4', 'L 3', 'D 1', 'R 4', 'D 1', 'L 5', 'R 2')> gives output <1>.
Test 1 passed: Input <('R 5', 'U 8', 'L 8', 'D 3', 'R 17', 'D 10', 'L 25', 'U 20')> gives output <36>.

Congratulations! Looks like you cracked it! Good job!


In [70]:
part_2_solution(puzzle_input)

2793